In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/userstoriestagged/UserStoriesTagged.csv
/kaggle/input/userstoriestagged/UserStoriesTagged2.csv
/kaggle/input/userstoriesall/UserStoriesTagged_Final_All_V2.csv
/kaggle/input/userstoriesall/UserStoriesTagged_Final_All.csv


In [3]:

import pandas as pd
import numpy as np

data=pd.read_csv("/kaggle/input/userstoriesall/UserStoriesTagged_Final_All_V2.csv",encoding="latin1")

TestData = data.sample(frac=0.30)
print("TestData Len:: " , len(TestData))
#print("TestData Index:: " , TestData.index)
TrainData = data.drop(TestData.index)
print("TrainData(without TestData) Len:: " , len(TrainData))

TestData Len::  5707
TrainData(without TestData) Len::  13316


In [4]:
data.tail(10)

,Story,Word,Tag
19013,story 6073,easily,non-Fun
19014,story 6074,better,non-Fun
19015,story 6074,most,non-Fun
19016,story 6075,better,non-Fun
19017,story 6075,most,non-Fun
19018,story 6089,wisely,non-Fun
19019,story 6127,easily,non-Fun
19020,story 6136,proper,non-Fun
19021,story 6136,properly,non-Fun
19022,story 6137,appropriately,non-Fun


In [5]:
#words = list(set(data["Word"].values))
words = list(set(TrainData["Word"].values))
n_words = len(words); n_words

test_words = list(set(TestData["Word"].values))

In [6]:

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        #print(agg_func)
        self.grouped = self.data.groupby("Story").apply(agg_func)
        #print(self.grouped)
        self.sentences = [s for s in self.grouped]
        #print(self.sentences)
    
    def get_next(self):
        try:
            #print(self.n_sent)
            #print(self.grouped)
            s = self.grouped["story {}".format(self.n_sent)]
            #print(s)
            self.n_sent += 1
            return s
        except:
            return None

In [7]:
getter = SentenceGetter(TrainData)
test_getter = SentenceGetter(TestData)
#print(test_getter)

In [8]:
sent = getter.get_next()
test_sent = test_getter.get_next()
print(sent)
print(test_sent)

[('Data user', 'User')]
[(' to have the 12-19-2017 deletions processed.', 'Fun')]


In [9]:
sentences = getter.sentences
test_sentences = test_getter.sentences
#print(sentences)
#print(test_sentences)

In [10]:
def word2features(sent, i):
    word = str(sent[i][0])
    postag = str(sent[i][1])
    #print("worddd: "+word)
    #print("postag: "+ postag )
    #print ("asdfsad:  " +  postag[:2])

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = str(sent[i-1][0])
        postag1 = str(sent[i-1][1])
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = str(sent[i+1][0])
        postag1 = str(sent[i+1][1])
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    #print(sent)
    #for token, label in sent:
    #    print(label)
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [11]:
X = [sent2features(s) for s in sentences]


In [ ]:
X

In [12]:
y = [sent2labels(s) for s in sentences]

In [13]:
def sent2labels2(sent):
    #print(sent)
    #for token, label in sent:
    #    print(label)
    return [[token, label] for token, label in sent]


In [56]:
#aa = [sent2labels2(s) for s in sentences]
#aa[0]

In [14]:
pip install sklearn-crfsuite

     |████████████████████████████████| 965 kB 287 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [16]:
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report

In [17]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [18]:
print(pred[0])
print(y[0])

['User']
['User']


In [20]:
pred

array([list(['User', 'Fun']), list(['User', 'Fun', 'Goal']),
       list(['User', 'Fun', 'Goal']), ..., list(['User', 'Fun']),
       list(['User', 'Fun']), list(['User', 'Fun'])], dtype=object)

In [19]:
report = flat_classification_report(y_pred=pred, y_true=y)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [20]:
print(report)

              precision    recall  f1-score   support

         Fun       1.00      1.00      1.00      4736
        Goal       1.00      1.00      1.00      2999
        User       1.00      1.00      1.00      4839
     non-Fun       1.00      1.00      1.00       742

    accuracy                           1.00     13316
   macro avg       1.00      1.00      1.00     13316
weighted avg       1.00      1.00      1.00     13316



# **NEW Evaluation##**
https://pypi.org/project/nervaluate/

In [21]:
pip install nervaluate

Note: you may need to restart the kernel to use updated packages.


In [22]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [23]:
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [24]:
test_x = [sent2features(s) for s in test_sentences]
print(len(test_x))
type(test_x)
test_x[:10]

4280


[[{'bias': 1.0,
   'word.lower()': ' to have the 12-19-2017 deletions processed.',
   'word[-3:]': 'ed.',
   'word[-2:]': 'd.',
   'word.isupper()': False,
   'word.istitle()': False,
   'word.isdigit()': False,
   'postag': 'Fun',
   'postag[:2]': 'Fu',
   'BOS': True,
   'EOS': True}],
 [{'bias': 1.0,
   'word.lower()': 'ui designer',
   'word[-3:]': 'ner',
   'word[-2:]': 'er',
   'word.isupper()': False,
   'word.istitle()': False,
   'word.isdigit()': False,
   'postag': 'User',
   'postag[:2]': 'Us',
   'BOS': True,
   'EOS': True}],
 [{'bias': 1.0,
   'word.lower()': ' find the relevant service and/or application type to initiate via the online portal.',
   'word[-3:]': 'al.',
   'word[-2:]': 'l.',
   'word.isupper()': False,
   'word.istitle()': False,
   'word.isdigit()': False,
   'postag': 'Goal',
   'postag[:2]': 'Go',
   'BOS': True,
   'EOS': True}],
 [{'bias': 1.0,
   'word.lower()': ' to have alfred to give me goals and missions that encourage me to do new activities an

In [25]:
test_y_true = [sent2labels(s) for s in test_sentences]
print(len(test_y_true))
#print(test_y_true)


4280


In [26]:
test_y_pred = crf.predict(test_x)
#print(test_y_pred)
#from sklearn.preprocessing import MultiLabelBinarizer
#mlb = MultiLabelBinarizer()
#new_y_pred = mlb.fit_transform(y_pred)
#new_y_test = mlb.fit_transform(y_test)


In [27]:
test_y_pred
test_y_true
len(test_y_true)
len(y[:1341])
len(test_x)

4280

In [29]:
def parse_test_data_ForEval(i):
    #TestData.tail(1)
    #i = 58
    #test_X
    #test_x = [sent2features(s) for s in test_sentences]
    p = crf.predict(test_x[i])
    p = np.argmax(p, axis=-1)
    true = np.argmax(test_y_true[i], -1)
    #print(test_X[650])
    #print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
    #print(30 * "=")
    #for w, t, pred in zip(test_x[i], true, p[0]):
        #print(w ," asdf ", t)
    #print(type(test_y_true[i]))
    return ([{'label': test_y_true[i][0] , "start":1 , "end": 1 }] ,
             [{'label': test_y_true[i-1][0] , "start":1 , "end": 1 }] )

In [30]:
tttt = [parse_test_data_ForEval(idx) for idx in range(len(test_x)) ] 
len(test_x)
len(tttt)
#print(tttt)
#type(tttt)
print("##################")
#[print(x[0]) for x in tttt]
truee = [x[0] for x in tttt]
#print(true)
print("##################")
#[print([x[1]]) for x in tttt]
predd = [x[1] for x in tttt]

/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


##################
##################


In [31]:
from nervaluate import Evaluator
evaluator = Evaluator(truee, predd, tags=['User', 'Fun', 'Goal','non-Fun'])

# Returns overall metrics and metrics for each tag

results, results_per_tag = evaluator.evaluate()

print(results)
#print(results_per_tag)

{'ent_type': {'correct': 3959, 'incorrect': 321, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 4280, 'actual': 4280, 'precision': 0.925, 'recall': 0.925, 'f1': 0.925}, 'partial': {'correct': 4280, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 4280, 'actual': 4280, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 'strict': {'correct': 3959, 'incorrect': 321, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 4280, 'actual': 4280, 'precision': 0.925, 'recall': 0.925, 'f1': 0.925}, 'exact': {'correct': 4280, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 4280, 'actual': 4280, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}


# **Training and Test Data**

In [ ]:
#pip install -U scikit-learn==0.16.1
!pip install scikit-learn==0.16.1

In [ ]:
y

In [44]:
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MultiLabelBinarizer
#new_y = MultiLabelBinarizer().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.5, random_state = 100)

In [45]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.23.2.


In [46]:
crf.fit(X, y)
#type(new_y)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [47]:
y_pred = crf.predict(X_test)
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
new_y_pred = mlb.fit_transform(y_pred)
new_y_test = mlb.fit_transform(y_test)

In [48]:
#mlb.classes_
list(mlb.classes_)
#mlb.inverse_transform(new_y_pred)
#y_pred

['Fun', 'Goal', 'User', 'non-Fun']

In [49]:
y_pred

[['User', 'Fun'],
 ['User', 'Fun', 'Goal', 'non-Fun'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun'],
 ['User', 'Fun', 'Goal', 'non-Fun'],
 ['User', 'Fun'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun'],
 ['User', 'Fun'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal', 'non-Fun'],
 ['User', 'Fun'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal', 'non-Fun'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun'],
 ['User', 'Fun'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun'],
 ['User', 'Fun'],
 ['User', 'Fun'],
 ['User', 'Fun', 'Goal', 'non-Fun'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun'],
 ['User', 'Fun', 'Goal'],
 ['User', 'Fun', 'non-Fun', 'non-Fun'],
 ['User', 'Fun'],
 ['

In [50]:
new_y_pred

array([[1, 0, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 0],
       ...,
       [1, 1, 1, 0],
       [1, 1, 1, 0],
       [0, 1, 1, 0]])

In [51]:
from sklearn.metrics import accuracy_score
accuracy =  accuracy_score(new_y_test, new_y_pred) * 100
print(accuracy)

100.0


In [52]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(new_y_test, new_y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1053
           1       1.00      1.00      1.00       702
           2       1.00      1.00      1.00      1084
           3       1.00      1.00      1.00       115

   micro avg       1.00      1.00      1.00      2954
   macro avg       1.00      1.00      1.00      2954
weighted avg       1.00      1.00      1.00      2954
 samples avg       1.00      1.00      1.00      2954

